In [84]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [85]:
# Import data
data = pd.read_csv('training_data_vt2025.csv')

In [86]:
def pre_processing(data):
    # Make copy of dataset
    data_processed = data.copy()

    # Create new summertime feature
    data_processed['is_summer'] = ((data_processed['month'] >= 3) & (data_processed['month'] <= 11)).astype(int)

    # Normalize calendar data using cosine encoding
    num_decimals = 6
    data_processed['day_of_week'] = np.round(np.cos(2 * np.pi * data_processed['day_of_week'] / 7.0), num_decimals)
    data_processed['month'] = np.round(np.cos(2 * np.pi * data_processed['month'] / 12), num_decimals)
    data_processed['hour_of_day'] = np.round(np.cos(2 * np.pi * data_processed['hour_of_day'] / 24.0), num_decimals)

    # Give target feature numerical values
    data_processed['increase_stock'] = data_processed['increase_stock'].replace({'high_bike_demand': 1, 'low_bike_demand': 0})

    # Create binary category of features
    data_processed['is_raining'] = (data_processed['precip'] != 0).astype(int)
    data_processed['is_snowing'] = (data_processed['snowdepth'] != 0).astype(int)
    data_processed['is_visible'] = (data_processed['visibility'] != 16).astype(int)

    # Drop columns
    data_processed = data_processed.drop(columns=['holiday', 'snow', 'summertime', 'snowdepth', 'precip', 'visibility', 'summertime'])

    # Standardize temp, dew and windspeed
    cols_to_standardize = ['temp', 'dew', 'windspeed']
    scaler_std = StandardScaler()
    data_processed[cols_to_standardize] = np.round(scaler_std.fit_transform(data_processed[cols_to_standardize]), num_decimals)

    # Normalize humidity and cloudcover
    cols_to_normalize = ['humidity', 'cloudcover']
    scaler_nor = MinMaxScaler()
    data_processed[cols_to_normalize] = np.round(scaler_nor.fit_transform(data_processed[cols_to_normalize]), num_decimals)
    return data_processed

In [87]:
new_data = pre_processing(data)

C:\Users\alexs\AppData\Local\Temp\ipykernel_21752\3347236774.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_processed['increase_stock'] = data_processed['increase_stock'].replace({'high_bike_demand': 1, 'low_bike_demand': 0})


In [88]:
# Save the processed data
new_data.to_csv("preprocessed_data.csv", index=False)

print("Preprocessing complete. File saved as preprocessed_data.csv")

Preprocessing complete. File saved as preprocessed_data.csv
